# Parallel MD Generation using Multiprocessing

This notebook generates molecular dynamics trajectories (`.xyz` files) for graphene nanoribbons.
It uses `multiprocessing` to run simulations for different parameters (Temperature, Length) in parallel, utilizing multiple CPU cores.

**Note:** The core simulation logic has been moved to `md_worker.py` to ensure compatibility with Python's `multiprocessing` on Windows.


In [2]:
import os
import multiprocessing
import numpy as np
import time

# Import the worker function from the external file
# This is required for multiprocessing on Windows
try:
    import md_worker
    from md_worker import worker_wrapper
except ImportError:
    print("Error: md_worker.py not found. Please ensure it is in the same directory.")


In [ ]:
# --- CONFIGURATION ---

# 1. Define Parameters
temperatures = [0, 50] # Kelvin
lengths = [10] # Unit cells

# 2. Simulation Steps
nsteps = 1000       # Total MD steps
dump_interval = 1   # Write to file every N steps

# 3. Output Directory
output_dir = "MD_files"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 4. Prepare Task List
# Format: (temp, length, filename, nsteps, interval)
tasks = []

print("Preparing simulation tasks:")
for T in temperatures:
    for L in lengths:
        filename = os.path.join(output_dir, f"md_T{T}_L{L}.xyz")
        # We need absolute path for safety in workers
        # abs_filename = os.path.abspath(filename) --- IGNORE ---
        
        # Tuple arguments for the worker_wrapper
        # args: (temp_K, ntile_length, filename, nsteps, dump_interval)
        task_args = (T, L, filename, nsteps, dump_interval)
        tasks.append(task_args)
        print(f"  - T={T}K, L={L} -> {filename}")

print(f"\nTotal tasks: {len(tasks)}")


Preparing simulation tasks:
  - T=0K, L=10 -> md_runs_parallel\md_T0_L10.xyz
  - T=50K, L=10 -> md_runs_parallel\md_T50_L10.xyz

Total tasks: 2


In [4]:
# --- EXECUTION ---

def run_parameter_sweep_parallel():
    # Determine number of processes
    # Leave one core free for OS responsiveness if possible
    num_processes = max(1, multiprocessing.cpu_count() - 2)
    print(f"Starting pool with {num_processes} processes...")
    
    start_time = time.time()
    
    # Use context manager for safety
    with multiprocessing.Pool(processes=num_processes) as pool:
        # map_async allows us to catch errors or track progress if implemented, 
        # but map is simpler and blocks until done.
        results = pool.map(worker_wrapper, tasks)
        
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"\nAll simulations completed in {duration:.2f} seconds.")
    return results

if __name__ == '__main__':
    # This check is good practice, though not strictly required inside the cell logic
    # unless exporting to script.
    completed_files = run_parameter_sweep_parallel()
    
    print("\nGenerated Files:")
    for f in completed_files:
        print(f"  {f}")


Starting pool with 2 processes...

All simulations completed in 1.15 seconds.

Generated Files:
  md_runs_parallel\md_T0_L10.xyz
  md_runs_parallel\md_T50_L10.xyz


In [4]:
# --- VERIFICATION ---
from ase.io import read

print("Verifying randomly selected file...")
if len(tasks) > 0:
    test_file = tasks[0][2] # Get filename of first task
    if os.path.exists(test_file):
        try:
            atoms = read(test_file, index=':')
            print(f"File: {test_file}")
            print(f"Number of frames: {len(atoms)}")
            print(f"Number of atoms: {len(atoms[0])}")
            print("Status: OK")
        except Exception as e:
            print(f"Failed to read {test_file}: {e}")
    else:
        print(f"File {test_file} was not created.")
else:
    print("No tasks defined.")


Verifying randomly selected file...
File: c:\Users\Asger\Documents\Bachelorkurser\10325\md_runs_parallel\md_T0_L10.xyz
Number of frames: 1001
Number of atoms: 200
Status: OK
